In [1]:
#there are 2 ways to recommend something
#colaborative filtering which uses user-user recomndation
#content based recommendation item-item recomndation
#netfix uses both of them to increase accuracy

In [2]:
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM

C:\Users\Sid\AppData\Local\conda\conda\envs\cten\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [23]:
#fetch data and format it
data =fetch_movielens(min_rating=4.0)
#constraint minimum rating is 4
#fetching data in form of csv file with 100k+ ratings of movies
#the data set contains 3 types of data 
#user-> userid , age , gender , occupation , zipcode 
#user_rating-> userid , movieid , rating , timestamp
#movie-> movieid , title, relesedate , videorelease , imdburl , genres
#the fetch_movielens cerates a 1+2D matrix containing user * movie 
#each intersection containg the rating given by the coresponding user
#the data is stored in the form of a dictionary which have 5 lables 
#'train', 'test', 'item_features', 'item_feature_labels', 'item_labels'
#bascically it is a 3D matrix in which the first layer is the type/classification layer

In [29]:
print(repr(data['train']))
print(repr(data['test']))
print(list(data),"\n\n")
print(data)

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>
['train', 'test', 'item_features', 'item_feature_labels', 'item_labels'] 


{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>, 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>, 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
	with 1682 stored elements in Compressed Sparse Row format>, 'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object), 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So 

In [35]:
#create model
model=LightFM(loss='warp')
#loss is a variable which checks for the loss between our trained model and our test model
#warp stands for weight approximate rank pairwise
#it uses gradient decent method 

In [36]:
model.fit(data['train'],epochs=30,num_threads=2)
#use to train data model takes the data set ot train on , no of epochs , no of threads

In [61]:
np.argsort(model.predict(3,np.arange(115))*-1)

array([ 99,  49,   6,   0,   8,  13,  92,  14, 110,  12,  18,   9,  24,
        95,  54,  11, 106,  31,  41,  23,  21,  52,  55,  56,  91,  32,
        19,  22,   5, 113, 107,   2,  97,  60,  88,   7, 108,  15,  78,
       112,  67,  85,  16,  30, 105,  58,  63,  69,  44,  33,  61,  48,
        82,  20,  38,  10,   1,  53,  94, 104,   3,  25,  75,  17,  37,
       102,  27,  76,  65,  28,  40,  59,  77, 103,  96,  36,  46,  93,
        81,  79,  90,   4,  68, 111,  35,  84,  70,  74,  62,  26, 114,
        80,  43,  83,  29,  50,  86,  72,  98,  73,  39,  89,  57,  34,
        45,  64,  47,  66,  87,  71, 109, 100,  42, 101,  51], dtype=int64)

In [54]:
#declaring a function which takes trained model data userid
def sample_recommendation(model,data,user_ids):
    n_users , n_items= data['train'].shape
    #shape function returns the rows into coulmn of the data set
    
    #generate recommendation for each user we input  
    for user_id in user_ids:
        #movies which are already liked by the user previously
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        #movies our model predict they like
        scores = model.predict(user_id, np.arange(n_items))
        #print(scores)
        #order them in most liked to least liked
        top_items = data['item_labels'][np.argsort(-scores)]

        #print out the results
        print ("user %s" % user_id)
        print ("         known positives:")

        for x in known_positives[:10]:
            print("              %s" % x)

        print("          Recommended:")

        for x in top_items[:10]:
            print("             %s" % x)
           
    
    

In [55]:
sample_recommendation(model, data , [2 , 71, 156])

[-0.27649835 -2.8295958  -1.3815906  ... -2.70033073 -2.79580498
 -2.80047917]
user 2
         known positives:
              Return of the Jedi (1983)
              Event Horizon (1997)
              Schindler's List (1993)
              Paradise Lost: The Child Murders at Robin Hood Hills (1996)
              Mother (1996)
              Cop Land (1997)
              Desperate Measures (1998)
              Boogie Nights (1997)
              Man Who Knew Too Little, The (1997)
              Apostle, The (1997)
          Recommended:
             L.A. Confidential (1997)
             English Patient, The (1996)
             Titanic (1997)
             Full Monty, The (1997)
             Boogie Nights (1997)
             Fargo (1996)
             Contact (1997)
             Chasing Amy (1997)
             Air Force One (1997)
             Good Will Hunting (1997)
[-2.0555501  -4.42897654 -4.54106092 ... -6.71838236 -6.55838442
 -6.73680401]
user 71
         known positives:
             